In [1]:
import os

In [2]:
data_dir ='cell_images\\cell_images'

In [3]:
os.listdir(data_dir)

['test', 'train']

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from matplotlib.image import imread

In [4]:
test_path= data_dir+'\\test\\'
train_path= data_dir+'\\train\\'

In [5]:
len(os.listdir(train_path+'parasitized'))
a=os.listdir(train_path+'parasitized')

In [6]:
train_path

'cell_images\\cell_images\\train\\'

In [9]:
dim1 = []
dim2 = []
for image_filename in os.listdir(test_path+'uninfected'):
    
    img = imread(test_path+'uninfected'+'\\'+image_filename)
    d1,d2,colors = img.shape
    dim1.append(d1)
    dim2.append(d2)

In [10]:
np.mean(dim1)

130.92538461538462

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
image_gen=ImageDataGenerator(rotation_range=20,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.1,zoom_range=0.1,horizontal_flip=True,fill_mode='nearest')

In [13]:
image_gen.flow_from_directory(train_path)

Found 24958 images belonging to 2 classes.


In [14]:
image_shape=(130,130,3)

In [15]:
from tensorflow.keras.models import Sequential

In [16]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout

In [17]:
model = Sequential()

model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add (MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add (MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add (MaxPool2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

In [18]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

In [20]:
early_stop=EarlyStopping(monitor='val_loss',patience=10)

In [21]:
train_image_gen=image_gen.flow_from_directory(train_path,target_size=image_shape[:2],color_mode='rgb',batch_size=16,class_mode='binary')

Found 24958 images belonging to 2 classes.


In [22]:
test_image_gen=image_gen.flow_from_directory(test_path,target_size=image_shape[:2],color_mode='rgb',batch_size=16,class_mode='binary',shuffle=False)

Found 2600 images belonging to 2 classes.


In [23]:
train_image_gen.class_indices

{'parasitized': 0, 'uninfected': 1}

In [24]:
from tensorflow.keras.models import load_model

In [25]:
model=load_model('malaria_detector.h5')

In [26]:
model.metrics_names

['loss', 'accuracy']

In [27]:
model.evaluate_generator(test_image_gen)

C:\Users\ZAIDI Mohamed Arysse\AppData\Local\Temp\ipykernel_36056\4011314882.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(test_image_gen)


[37.07537841796875, 0.8734615445137024]

In [28]:
pred=model.predict_generator(test_image_gen)

C:\Users\ZAIDI Mohamed Arysse\AppData\Local\Temp\ipykernel_36056\1813018069.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred=model.predict_generator(test_image_gen)


In [29]:
predictions=pred>0.5

In [30]:
from sklearn.metrics import confusion_matrix,classification_report

In [31]:
print(classification_report(test_image_gen.classes,predictions))

              precision    recall  f1-score   support

           0       0.81      0.98      0.89      1300
           1       0.97      0.77      0.86      1300

    accuracy                           0.87      2600
   macro avg       0.89      0.87      0.87      2600
weighted avg       0.89      0.87      0.87      2600

